## Task: Build a Campus FAQ Chatbot using RAG

### Objective:
Learn how Retrieval-Augmented Generation (RAG) works by building a small chatbot that answers questions about your college using vector embeddings and a mini vector database.

#### Step 0: Setup

1. Install required packages:

In [1]:
pip install streamlit sentence-transformers faiss-cpu numpy

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/2a/38/991bbf9fa3ed3d9c8e69265fc449bdaade8131c7f0f750dbd388c3c477dc/streamlit-1.50.0-py3-none-any.whl.metadata
  Using cached streamlit-1.50.0-py3-none-any.whl.metadata (9.5 kB)
  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/48/21/4670d03ab8587b0ab6f7d5fa02a95c3dd6b1f39d0e40e508870201f3d76c/sentence_transformers-5.1.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/c9/b8/6911de6b8fdcfa76144680c2195df6ce7e0cc920a8be8c5bbd2dfe5e3c37/faiss_cpu-1.12.0-cp312-cp312-win_amd64.whl.metadata
  Using cached faiss_cpu-1.12.0-cp312-cp312-win_amd64.whl.metadata (5.2 kB)
  Obtaining dependency information for altair!=5.4.0,!=5.4.1,<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc9


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Step 1: Prepare the Data

Task: Create a small FAQ dataset with at least 5 Q&A pairs.
Example:

Q: When does the library open?
A: The library opens at 8 AM and closes at 8 PM.

Checkpoint:

Students should have a list of questions and answers ready.

#### Step 2: Split Text into Chunks

Task: Split your FAQ into separate lines to treat each Q&A as a chunk.

In [2]:
faq_text = """
Q:What’s the difference between espresso and drip coffee? A:Espresso is made by forcing hot water through finely-ground coffee at high pressure, producing a strong, concentrated shot with crema on top. Drip coffee lets water slowly pass through coarser grounds, giving a lighter flavor.

How is decaf coffee made? Decaf coffee is made by removing about 97% of the caffeine from green coffee beans before roasting. Methods include the Swiss Water Process or solvent extraction.

What are the best beans for cold brew? Medium to dark roast beans work best for cold brew because they give smooth, chocolatey flavors when steeped for long hours.

Why do baristas use milk frothers? Milk frothers create microfoam that gives cappuccinos and lattes their creamy texture and blends well with espresso.

What is the origin of cappuccino? The cappuccino originated in Italy and was named after the Capuchin monks because its color resembled their robes.

How much caffeine is in a latte? A typical 12-oz latte contains about 60–80 mg of caffeine depending on the beans and shot size.

What’s the difference between Arabica and Robusta beans? Arabica beans are smoother and sweet, while Robusta beans are stronger, more bitter, and have more caffeine.

Can coffee help with focus? Yes! Caffeine blocks adenosine, a brain chemical that makes you sleepy, and boosts dopamine and alertness.

What is latte art? Latte art is the creative design made when pouring steamed milk into espresso, forming shapes like hearts or leaves.

How should coffee beans be stored? Keep beans in an airtight container away from light, heat, and moisture to keep them fresh longer.
"""


In [3]:
lines = [line.strip() for line in faq_text.split("\n") if line.strip()]


Checkpoint:

Ensure each Q&A is a separate element in a Python list.

In [4]:
lines

['Q:What’s the difference between espresso and drip coffee? A:Espresso is made by forcing hot water through finely-ground coffee at high pressure, producing a strong, concentrated shot with crema on top. Drip coffee lets water slowly pass through coarser grounds, giving a lighter flavor.',
 'How is decaf coffee made? Decaf coffee is made by removing about 97% of the caffeine from green coffee beans before roasting. Methods include the Swiss Water Process or solvent extraction.',
 'What are the best beans for cold brew? Medium to dark roast beans work best for cold brew because they give smooth, chocolatey flavors when steeped for long hours.',
 'Why do baristas use milk frothers? Milk frothers create microfoam that gives cappuccinos and lattes their creamy texture and blends well with espresso.',
 'What is the origin of cappuccino? The cappuccino originated in Italy and was named after the Capuchin monks because its color resembled their robes.',
 'How much caffeine is in a latte? A ty

#### Step 3: Create Embeddings

Task: Convert each line to a vector using SentenceTransformer.

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(lines)

In [5]:
from sentence_transformers import SentenceTransformer

C:\Users\Sandhya Siva kumar\PycharmProjects\Micro Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(lines)

#### Step 4: Build the FAISS Index

Task: Store all embeddings in a FAISS vector database.

In [14]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

#### Step 5: Query the Database
Task: Take a user question, convert it to a vector, and find the most relevant FAQ line.

In [16]:
user_question = ("Best coffee?")
q_emb = model.encode([user_question])
D, I = index.search(np.array(q_emb), k=1)
print("Answer:", lines[I[0][0]])

Answer: What are the best beans for cold brew? Medium to dark roast beans work best for cold brew because they give smooth, chocolatey flavors when steeped for long hours.


#### Step 6: Make it Interactive with Streamlit
Task: Use Streamlit to create a simple chatbot UI.

In [17]:
import pickle


# Save the already existing objects
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

import faiss, numpy as np, pickle

faiss.write_index(index, "index.faiss")   # save FAISS index safely
np.save("lines.npy", np.array(lines, dtype=object))  # save lines
# model: reload by name in app, so no need to pickle it
# (you can keep: model_name="all-MiniLM-L6-v2")

with open("index.pkl", "wb") as f:
    pickle.dump(index, f)

with open("lines.pkl", "wb") as f:
    pickle.dump(lines, f)

In [10]:
import streamlit as st

st.title("Course FAQ Chatbot")
user_question = st.text_input("Ask your question:")
if user_question:
    q_emb = model.encode([user_question])
    D, I = index.search(np.array(q_emb), k=1)
    st.write("Answer:", lines[I[0][0]])

2025-10-07 23:54:39.547 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23:54:39.609 
  command:

    streamlit run C:\Users\Sandhya Siva kumar\PycharmProjects\AML-3303\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-07 23:54:39.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23:54:39.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23:54:39.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23:54:39.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23:54:39.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-07 23

#### Step 7: Reflection

Questions for students:

How does the chatbot “understand” the question?

What happens if the user asks something not in the FAQ?

How could you improve this system to handle more questions or longer documents?